<a href="https://colab.research.google.com/github/Dm1stry/NeuralNetworks/blob/main/MyFirstNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
class Layer:
  def __init__(self):
    self.input = None
    self.output = None
  
  def forwardfeed(self, input):
    raise NotImplementedError

  def backwardfeed(self, output_error, learning_rate):
    raise NotImplementedError

In [ ]:
class FCLayer(Layer):
  def __init__(self, input_size, output_size):
    self.weights = np.random.rand(input_size, output_size) - 0.5
    self.bias = np.random.rand(1, output_size) - 0.5

  def forwardfeed(self, input_data):
    self.input = input_data
    self.output = self.input @ self.weights + self.bias
    return self.output

  def backwardfeed(self, output_error, learning_rate):
    input_error = output_error @ np.transpose(self.weights)
    weights_error = np.transpose(self.input) @ output_error
    bias_error = output_error

    self.weights -= learning_rate * weights_error
    self.bias -= learning_rate * bias_error
    return input_error

In [ ]:
class ActivationLayer(Layer):
  def __init__(self, activation_func, activation_deriv):
    self.activation_func = activation_func
    self.activation_deriv = activation_deriv

  def forwardfeed(self, input_data):
    self.input = input_data
    self.output = self.activation_func(input_data)

    return self.output

  def backwardfeed(self, output_error, learning_rate):
    return self.activation_deriv(self.input) * output_error

In [ ]:
def tanh(x):
  return np.tanh(x)

def tanh_deriv(x):
  return 1 / (np.cosh(x) ** 2)

In [ ]:
def loss_mse(y_true, y_predicted):
  return np.mean(np.power(y_true - y_predicted, 2))

def loss_mse_deriv(y_true, y_predicted):
  return 2 * ( y_predicted - y_true) / y_true.size

In [ ]:
class NeuralNetwork:
  def __init__(self, loss = None, loss_deriv = None):
    self.layers = []
    self.use(loss, loss_deriv)
  
  def add_layer(self, layer):
    self.layers.append(layer)

  def use(self, loss, loss_deriv):
    self.loss = loss
    self.loss_deriv = loss_deriv

  def predict(self, x):
    result = []
    for sample in range(len(x)):
      output = x[sample]
      for layer in self.layers:
        output = layer.forwardfeed(output)
      result.append(output)
    return np.array(result)

  def fit(self, x, y_true, epochs=1000, learning_rate=0.1):
    for epoch in range(1, epochs+1):
      err = 0
      for sample in range(len(x)):
        y = x[sample]
        for layer in self.layers:
          y = layer.forwardfeed(y)
        
        err += self.loss(y_true[sample], y)

        error = self.loss_deriv(y_true[sample], y)

        for layer in reversed(self.layers):
          error = layer.backwardfeed(error, learning_rate)
      if err < 0.000001:
        break
      if epoch % 100 == 0:
        err /= len(x)
        print("epoch %d/%d  error=%f" % (epoch, epochs, err))

In [ ]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

net = NeuralNetwork(loss_mse, loss_mse_deriv)
net.add_layer(FCLayer(2, 5))
net.add_layer(ActivationLayer(tanh, tanh_deriv))
net.add_layer(FCLayer(5, 3))
net.add_layer(ActivationLayer(tanh, tanh_deriv))
net.add_layer(FCLayer(3, 1))

net.fit(x_train, y_train, learning_rate=0.1)

out = np.array(net.predict(x_train))
print("y_train: ", y_train, '\nNeuarlNetworkOutput: ', out, '\nfinal error: ', loss_mse(out, y_train))

epoch 100/1000  error=0.279481
y_train:  [[[0]]

 [[1]]

 [[1]]

 [[0]]] 
NeuarlNetworkOutput:  [[[ 3.79301948e-04]]

 [[ 1.00027625e+00]]

 [[ 1.00011447e+00]]

 [[-8.31638679e-06]]] 
final error:  5.833862518132556e-08


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import pandas as pd

data = datasets.load_breast_cancer()
ndata = normalize(data.data)

data_train, data_test, target_train, target_test = train_test_split(ndata, data.target)

cancer_net = NeuralNetwork(loss_mse, loss_mse_deriv)

cancer_net.add_layer(FCLayer(30, 45))
cancer_net.add_layer(ActivationLayer(tanh, tanh_deriv))
cancer_net.add_layer(FCLayer(45, 20))
cancer_net.add_layer(ActivationLayer(tanh, tanh_deriv))
cancer_net.add_layer(FCLayer(20, 1))

In [ ]:
def reformat(data):
  if len(data.shape) == 1:
    row_len = 1
  else:
    row_len = data.shape[1]
  reformated = np.zeros((data.shape[0], 1, row_len))
  for i, row in enumerate(data):
    reformated[i][0] = row
  return reformated

In [ ]:
cancer_net.fit(reformat(data_train), reformat(target_train), epochs=10000)

epoch 100/10000  error=0.083906
epoch 200/10000  error=0.079813
epoch 300/10000  error=0.076575
epoch 400/10000  error=0.075841
epoch 500/10000  error=0.074997
epoch 600/10000  error=0.072676
epoch 700/10000  error=0.070887
epoch 800/10000  error=0.069068
epoch 900/10000  error=0.067053
epoch 1000/10000  error=0.065110
epoch 1100/10000  error=0.062219
epoch 1200/10000  error=0.060022
epoch 1300/10000  error=0.059085
epoch 1400/10000  error=0.058414
epoch 1500/10000  error=0.057943
epoch 1600/10000  error=0.057609
epoch 1700/10000  error=0.057368
epoch 1800/10000  error=0.057176
epoch 1900/10000  error=0.057029
epoch 2000/10000  error=0.056854
epoch 2100/10000  error=0.056813
epoch 2200/10000  error=0.056930
epoch 2300/10000  error=0.057079
epoch 2400/10000  error=0.057067
epoch 2500/10000  error=0.057934
epoch 2600/10000  error=0.056915
epoch 2700/10000  error=0.056632
epoch 2800/10000  error=0.056522
epoch 2900/10000  error=0.056549
epoch 3000/10000  error=0.056608
epoch 3100/10000  e

In [ ]:
out = cancer_net.predict(data_test).astype(int)
df = pd.DataFrame(data={'y_predict' : out.reshape(143), 'y_true' : target_test.reshape(143)})
loss_mse(out, reformat(target_test))

0.16783216783216784